In [60]:
#given .log file:
#     create datframe of points
#     (array i = source i + 1)
#     dataframe: key:timestamp -> 2D array[[id, tag, x, y,z, activity]
#                                    [id, tag, x, y,z, activity]
#                                    [id, tag, x, y,z, activity]
#                                    [id, tag, x, y,z, activity]]
#     in dataframe:
#     if(id of all source == 0):
#         delete datapoint
#     else
#         convert timestamp to time
#  
#     return dataframe

In [61]:
#merge log files with same starting time

In [68]:
import pandas as pd
import re 
import json
import datetime
import os
import time
import glob

In [93]:
def timeExtract(filename):
    with open(filename, 'rb') as f:
        # Start counting from the last byte
        counter = 1
        # Go to the 2nd byte before the end of the last line
        f.seek(-2, 2) 
        while f.read(1) != b'\n':
            f.seek(-2, 1)
            counter=counter+1
        endTime_line = f.readline().decode()
        # Go to the 2nd byte before the end of the last second line
        f.seek(-counter-2, 2)
        while f.read(1) != b'\n':
            f.seek(-2, 1)
        startTime_line = f.readline().decode()

    return [startTime_line, endTime_line]


def durationinMicroseconds(filename):
    startTime = timeExtract(filename)[0].split()[2:]
    endTime = timeExtract(filename)[1].split()[2:]
    startTimeStr = startTime[0] + ' ' + startTime[1]
    endTimeStr = endTime[0] + ' ' + endTime[1]
    T1 = datetime.datetime.strptime(startTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    T2 = datetime.datetime.strptime(endTimeStr, '%Y-%m-%d %H:%M:%S.%f')
    delta = T2-T1
    duration = delta.seconds*1000000 + delta.microseconds
    
    return duration, T1, T2



def extractDirectionalities(filename):
    with open(filename, 'r') as f:
        text = f.read()
        # Use repex to store blocks of data into a list
    data = re.split('(?<=})\n(?={)', text) 
        # Delete the time info from the last data block
    tmp = data[-1][:(data[-1].rfind("}")+1)]
    data[-1] = tmp
        

    srcList = [json.loads(block)["src"] for block in data]
    
    source= dict()
    df = pd.DataFrame(columns = ['timestamp', 'time', 'source'])
    
    #Used for calculating timestamps -> time
    duration, startTime, endTime = durationinMicroseconds(filename)
    t = duration/len(data)
    
     #convwert timestamp to time 
            #crest 2darray

            #put time in df
    
    index = 1.0
    for block in srcList:
        if block[0]["id"] != 0 or block[1]["id"] != 0 or block[2]["id"] != 0 or block[3]["id"] != 0:
            timesince = datetime.datetime.now() - startTime
            time = timesince.total_seconds() + (index - 1.0) * t
            array = ([block[0]["id"], block[0]["tag"], block[0]["x"], block[0]["y"], block[0]["z"], block[0]["activity"]], 
                     [block[1]["id"], block[1]["tag"], block[1]["x"], block[1]["y"], block[1]["z"], block[1]["activity"]],
                     [block[2]["id"], block[2]["tag"], block[2]["x"], block[2]["y"], block[2]["z"], block[2]["activity"]], 
                     [block[3]["id"], block[3]["tag"], block[3]["x"], block[3]["y"], block[3]["z"], block[3]["activity"]])
            df2 = pd.DataFrame({"timestamp": index, "time":datetime.datetime.fromtimestamp(time).strftime("%A, %B %d, %Y %I:%M:%S"), "source": array})
            df = df.append(df2)
            index = index + 1.0
            
    print(df)
            
            
        
        
            

In [94]:
filename = "/Users/brian_wangst/Desktop/MLR/data/cSST_2019-09-25_04_55_03.log"
extractDirectionalities(filename)

    timestamp                                  time  \
0         1.0  Saturday, February 28, 1970 01:30:08   
1         1.0  Saturday, February 28, 1970 01:30:08   
2         1.0  Saturday, February 28, 1970 01:30:08   
3         1.0  Saturday, February 28, 1970 01:30:08   
0         2.0  Saturday, February 28, 1970 03:43:26   
1         2.0  Saturday, February 28, 1970 03:43:26   
2         2.0  Saturday, February 28, 1970 03:43:26   
3         2.0  Saturday, February 28, 1970 03:43:26   
0         3.0  Saturday, February 28, 1970 05:56:44   
1         3.0  Saturday, February 28, 1970 05:56:44   
2         3.0  Saturday, February 28, 1970 05:56:44   
3         3.0  Saturday, February 28, 1970 05:56:44   
0         4.0  Saturday, February 28, 1970 08:10:02   
1         4.0  Saturday, February 28, 1970 08:10:02   
2         4.0  Saturday, February 28, 1970 08:10:02   
3         4.0  Saturday, February 28, 1970 08:10:02   
0         5.0  Saturday, February 28, 1970 10:23:20   
1         